# 1. Preprocess-Stage1-Group-All-TrainData-KFold

## Run name

In [1]:
import time
project_name = 'Google-LandMark-Rec2019'
step_name = '1-Preprocess-Stage1-Group-All-TrainData-KFold'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData-KFold_20190504-094152


## Important params

In [2]:
import multiprocessing
import random

cpu_amount = multiprocessing.cpu_count()
random_num = random.randint(0, 10000)
# train_tar_count = 500
train_data_ratio = 0.95

# print('train_tar_count:', train_tar_count)
print('cpu_amount: ', cpu_amount)
print('random_num:', random_num)
print(train_data_ratio)

cpu_amount:  2
random_num: 7855
0.95


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display
from sklearn.model_selection import train_test_split, KFold

import os, sys, gc, math, shutil, zipfile, pickle, h5py, re, tarfile
import urllib, xlsxwriter
from tqdm import tqdm
import hashlib

## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

md5sum_folder = os.path.join(input_folder, 'md5sum')
if not os.path.exists(md5sum_folder):
    os.mkdir(md5sum_folder)
    print('create folder:', md5sum_folder)
    
train_tar_folder = os.path.join(input_folder, 'train_tar')
if not os.path.exists(train_tar_folder):
    os.mkdir(train_tar_folder)
    print('create folder:', train_tar_folder)
    
train_untar_folder = os.path.join(input_folder, 'train_untar')
if not os.path.exists(train_untar_folder):
    os.mkdir(train_untar_folder)
    print('create folder:', train_untar_folder)
    
all_image_folder = os.path.join(input_folder, 'all_image')
if not os.path.exists(all_image_folder):
    os.mkdir(all_image_folder)
    print('create folder:', all_image_folder)
    
# data_train_folder = os.path.join(input_folder, 'data_train')
# if not os.path.exists(data_train_folder):
#     os.mkdir(data_train_folder)
#     print('create folder:', data_train_folder)
    
# data_val_folder = os.path.join(input_folder, 'data_val')
# if not os.path.exists(data_val_folder):
#     os.mkdir(data_val_folder)
#     print('create folder:', data_val_folder)

In [5]:
!pwd
!ls ./input -hl

/data/landmark-recognition-2019
total 1.7G
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_00
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_01
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_02
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_03
drwxr-xr-x     2 root   root   4.0K May  4 08:28 50000_04
drwxr-xr-x     2 root   root   160M May  2 21:14 all_image
drwxr-xr-x 34162 root   root   824K May  4 08:59 data_val_00
drwxr-xr-x 34198 root   root   816K May  4 09:14 data_val_01
drwxr-xr-x 34266 root   root   808K May  4 09:23 data_val_02
drwxr-xr-x 34157 root   root   808K May  4 09:33 data_val_03
drwxr-xr-x  6119 root   root   132K May  4 09:34 data_val_04
drwxr-xr-x     2 root   root   4.0K May  4 08:29 demo_01
drwxr-xr-x     2 root   root   4.0K May  4 08:29 demo_02
drwxr-xr-x     2 root   root   4.0K May  4 08:29 demo_03
drwxr-xr-x     2 root   root   4.0K May  4 08:29 demo_04
-rw-rw-r--     1 ubuntu ubuntu 108M May  1 17:18 google-landmarks-dataset.z

In [6]:
!ls ./input/all_image/ -U|head -5

e42936e3627d8989.jpg
96d0f4fec18d8c84.jpg
02fd1c4af95765fd.jpg
37bdd02aebba82a6.jpg
b732241b0be6a043.jpg
ls: write error: Broken pipe


In [7]:
# !wget https://s3.amazonaws.com/google-landmark/metadata/train.csv
# !wget https://s3.amazonaws.com/google-landmark/metadata/train_attribution.csv

In [8]:
train_csv = os.path.join(input_folder, 'train.csv')
test_csv = os.path.join(input_folder, 'test.csv')
boxes_split1_csv = os.path.join(input_folder, 'google_landmark_boxes_split1.csv')
boxes_split2_csv = os.path.join(input_folder, 'google_landmark_boxes_split2.csv')
recognition_sample_submission_csv = os.path.join(input_folder, 'recognition_sample_submission.csv')

print(train_csv)
print(test_csv)
print(boxes_split1_csv)
print(boxes_split2_csv)
print(recognition_sample_submission_csv)


/data/landmark-recognition-2019/input/train.csv
/data/landmark-recognition-2019/input/test.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split1.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split2.csv
/data/landmark-recognition-2019/input/recognition_sample_submission.csv


## Preview csv

In [9]:
%%time
train_df = pd.read_csv(train_csv, engine='python', nrows=None)
display(train_df.shape, train_df.head(2))

(4132914, 3)

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169


CPU times: user 17.9 s, sys: 1.74 s, total: 19.6 s
Wall time: 36.4 s


In [10]:
data_train_df, data_val_df = train_test_split(train_df, train_size=train_data_ratio, random_state=random_num, shuffle=True)
display(data_train_df.shape, data_train_df.head(2))
display(data_val_df.shape, data_val_df.head(2))

/root/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(3926268, 3)

,id,url,landmark_id
1370931,e2760b8756e3abc7,https://upload.wikimedia.org/wikipedia/commons...,95346
3600979,f719ad140137e35a,https://upload.wikimedia.org/wikipedia/commons...,192746


(206646, 3)

,id,url,landmark_id
1322202,0b709998bdfd7cf8,https://upload.wikimedia.org/wikipedia/commons...,183360
1740531,f718c1ddb25dd3c3,https://upload.wikimedia.org/wikipedia/commons...,110247


## Split df firstly, then group all categories by creating soft link

In [11]:
def split_group(df, all_image_folder, data_set_name='demo', split_size=50000):
    split_amount = math.ceil(df.shape[0]/split_size)
    print('split_amount:', split_amount)
        
    data_item_invalid_count = 0
    data_item_softlink_count = 0
    for i, start_index in enumerate(range(0, df.shape[0], split_size)):
        print(i, start_index, end='\t')
        split_df = df.loc[df.index[start_index: (start_index+split_size)]]
        print(split_df.shape, end='\t')

        data_item_folder = os.path.join(input_folder, '%s_%02d' % (data_set_name, i))
        if not os.path.exists(data_item_folder):
            os.mkdir(data_item_folder)
            print('Folder crated:', data_item_folder, end='\t')
        for idx, row in split_df.iterrows():
            src_image_path = os.path.join(all_image_folder, '%s.jpg' % row['id'])
        #     print(src_image_path)
            if not os.path.exists(src_image_path):
                data_item_invalid_count += 1
                continue
        #     print(src_image_path)
            target_image_folder = os.path.join(data_item_folder, '%06d' % row['landmark_id'])
        #     print(target_image_folder)
            if not os.path.exists(target_image_folder):
                os.mkdir(target_image_folder)
        #         print('Folder created:', target_image_folder)
            target_image_path = os.path.join(target_image_folder, '%s.jpg' % row['id'])
            if not os.path.exists(target_image_path):
                data_item_softlink_count += 1
                os.symlink(src_image_path, target_image_path)
            if data_item_softlink_count % 1000 == 0:
                print('-', end='')
#             if data_item_softlink_count % 10000 == 0:
#                 print('|')

        print('data_item_invalid_count:', data_item_invalid_count, end='\t')
        print('data_item_softlink_count:', data_item_softlink_count)
        print('*' * 40)
    print('*' * 80)


In [12]:
split_group(data_train_df, all_image_folder, 'data_train')

split_amount: 79
0 0	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_00	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 50000
****************************************
1 50000	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_01	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 100000
****************************************
2 100000	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_02	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 150000
****************************************
3 150000	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_03	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 200000
****************************************
4 200000	(50000, 3)	Folder crated: /da

68 3400000	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_68	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 3450000
****************************************
69 3450000	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_69	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 3500000
****************************************
70 3500000	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_70	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 3550000
****************************************
71 3550000	(50000, 3)	Folder crated: /data/landmark-recognition-2019/input/data_train_71	--------------------------------------------------data_item_invalid_count: 0	data_item_softlink_count: 3600000
****************************************
72 3600000	(50000, 3)	Folder crated:

In [13]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name)

Time elapsed: 51766.1s
Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData-KFold_20190504-094152
